In [1]:
import sys
sys.path.append('../')
import time
from datasets import load_dataset
import torch
torch.cuda.empty_cache()
import transformers
transformers.logging.set_verbosity_warning()
from transformers import AutoTokenizer
from src.frameworks import TruncatedMC

c:\Users\AronQ\.conda\envs\flenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("yelp_review_full")

Found cached dataset yelp_review_full (C:/Users/AronQ/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)
100%|██████████| 2/2 [00:00<00:00,  6.25it/s]


In [3]:
dataset["train"][2]

{'label': 3,
 'text': "Been going to Dr. Goldberg for over 10 years. I think I was one of his 1st patients when he started at MHMG. He's been great over the years and is really all about the big picture. It is because of him, not my now former gyn Dr. Markoff, that I found out I have fibroids. He explores all options with you and is very patient and understanding. He doesn't judge and asks all the right questions. Very thorough and wants to be kept in the loop on every aspect of your medical health and your life."}

In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [6]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at C:\Users\AronQ\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-aad1af4c7095bfa1.arrow
Loading cached processed dataset at C:\Users\AronQ\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-29f27748f0b54d01.arrow


In [7]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(5))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(5))

Loading cached shuffled indices for dataset at C:\Users\AronQ\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-11a7619c6a3c070f.arrow
Loading cached shuffled indices for dataset at C:\Users\AronQ\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-3c5c2a245be1b332.arrow


In [8]:
type(small_train_dataset)

datasets.arrow_dataset.Dataset

In [9]:
small_train_dataset

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5
})

In [10]:
type(small_train_dataset['text'])

list

The following cell output should contain a warning like: "Some weights of the model checkpoint at bert-base-cased were not used when initializing...".

This is OK as we are not using the pooler layer in order to compute the loss.

In [11]:
dshap = TruncatedMC(train_dataset=small_train_dataset, X_train=small_train_dataset['text'], X_test=small_eval_dataset['text'], test_dataset=small_eval_dataset, model_family='bert-base-cased', num_labels=5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Print the initial accuracy

In [12]:
dshap.random_score

0.4

In [13]:
time.asctime()

'Wed May  3 01:03:05 2023'

In [14]:
dshap.run(save_every=100, err=1, do_loo=True, do_tmc=True, do_gshap=False)

Calculate leave-one-out


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

{'train_runtime': 16.5826, 'train_samples_per_second': 0.603, 'train_steps_per_second': 0.121, 'train_loss': 1.646740436553955, 'epoch': 2.0}
{'eval_loss': 1.9394981861114502, 'eval_accuracy': 0.4, 'eval_runtime': 3.279, 'eval_samples_per_second': 1.525, 'eval_steps_per_second': 0.305, 'epoch': 2.0}


  0%|          | 0/5 [00:00<?, ?it/s]Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initi

{'train_runtime': 13.0151, 'train_samples_per_second': 0.615, 'train_steps_per_second': 0.154, 'train_loss': 1.337770700454712, 'epoch': 2.0}


...

{'train_runtime': 13.965, 'train_samples_per_second': 0.573, 'train_steps_per_second': 0.143, 'train_loss': 1.4319584369659424, 'epoch': 2.0}


100%|██████████| 5/5 [01:33<00:00, 18.71s/it]

{'eval_loss': 2.1640307903289795, 'eval_accuracy': 0.2, 'eval_runtime': 3.639, 'eval_samples_per_second': 1.374, 'eval_steps_per_second': 0.275, 'epoch': 2.0}



Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at 

{'train_runtime': 16.598, 'train_samples_per_second': 0.602, 'train_steps_per_second': 0.12, 'train_loss': 1.4257745742797852, 'epoch': 2.0}
{'eval_loss': 1.8735370635986328, 'eval_accuracy': 0.4, 'eval_runtime': 3.351, 'eval_samples_per_second': 1.492, 'eval_steps_per_second': 0.298, 'epoch': 2.0}
{'eval_loss': 1.7414395809173584, 'eval_accuracy': 0.4, 'eval_runtime': 3.529, 'eval_samples_per_second': 1.417, 'eval_steps_per_second': 0.283, 'epoch': 2.0}
{'eval_loss': 2.403810977935791, 'eval_accuracy': 0.0, 'eval_runtime': 3.554, 'eval_samples_per_second': 1.407, 'eval_steps_per_second': 0.281, 'epoch': 2.0}
{'eval_loss': 2.002265691757202, 'eval_accuracy': 0.2, 'eval_runtime': 3.368, 'eval_samples_per_second': 1.485, 'eval_steps_per_second': 0.297, 'epoch': 2.0}
{'eval_loss': 1.79889714717865, 'eval_accuracy': 0.4, 'eval_runtime': 3.523, 'eval_samples_per_second': 1.419, 'eval_steps_per_second': 0.284, 'epoch': 2.0}
{'eval_loss': 2.403810977935791, 'eval_accuracy': 0.0, 'eval_runtime

In [ ]:
time.asctime()

In [ ]:
dshap.vals_loo

In [ ]:
dshap.vals_tmcshap